# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.
Notice that `PlaneWaveBasis` defaults to just using the ``Γ``-point
if no ``k``-point options are provided.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5)

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.123538e+02     1.663824e+00
 * time: 0.09135198593139648
     1     1.114924e+01     8.974484e-01
 * time: 0.9727389812469482
     2    -1.181448e+01     1.092021e+00
 * time: 1.129668951034546
     3    -3.360739e+01     8.273377e-01
 * time: 1.27056884765625
     4    -4.727954e+01     5.262481e-01
 * time: 1.4315869808197021
     5    -5.674310e+01     1.894482e-01
 * time: 1.586332082748413
     6    -5.975647e+01     1.254155e-01
 * time: 1.7030949592590332
     7    -6.087355e+01     5.783431e-02
 * time: 1.8328289985656738
     8    -6.129232e+01     5.797227e-02
 * time: 1.9430139064788818
     9    -6.158587e+01     3.722474e-02
 * time: 2.061706066131592
    10    -6.182224e+01     2.897894e-02
 * time: 2.172581911087036
    11    -6.201136e+01     2.337912e-02
 * time: 2.2908668518066406
    12    -6.208541e+01     1.849311e-02
 * time: 2.40077805519104
    13    -6.215793e+01     1.684097e-02
 * time: 2.51869797706604
  

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671067
    AtomicLocal         -18.8557708
    AtomicNonlocal      14.8522674
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485376 
    Xc                  -19.3336821

    total               -62.261666461685
